![heading2](https://user-images.githubusercontent.com/72619946/98711135-3a717c00-234a-11eb-91f5-a218f35de775.jpeg)
# WELCOME! 

### This is the coding playground for the third D(3) Digitial Deep Dive 
### Novermber 3rd 2020
Questions? Feel free to reach out to Christian Lynn
christian_lynn@baxter.com

In [ ]:
#Usual Imports
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from scipy.stats import norm, skew #for some statistics
from scipy import stats

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
#resetting default plot shape
plt.tight_layout(False)
plt.rcParams.update({'text.color' : "r",
                     'axes.labelcolor' : "r"})
plt.rcParams['xtick.color'] = 'r'
plt.rcParams['ytick.color'] = 'r'
sns.set(rc={'figure.figsize':(10,8)})

## **Starting this Analysis by reading in 2 subsets of data:**
the data collected by the end of session 2, and the full raw data

In [ ]:
#RAW DATA HERE
url = 'https://raw.githubusercontent.com/ChristianRLynn/DataSets/main/train.csv'
raw = pd.read_csv(url, error_bad_lines=False); raw.head(10)

In [ ]:
#EDITED DATA LETS SEE HOW WE DID
url2 = 'https://raw.githubusercontent.com/ChristianRLynn/DataSets/main/new_data_intermediate_V2.csv'
part2_data = pd.read_csv(url2, error_bad_lines=False); part2_data.tail(10)

# The two datasets are below, and using the .shape funtion I can see how big each dataset is in the shape (rows,columns). 
Notice that part2_data is actually bigger than the raw dataset, those are our created features!

In [ ]:
raw.shape

In [ ]:
part2_data.shape

In [ ]:
part2_data.columns

In [ ]:
#Looking at column data, super helpfull to see our features:
raw.columns

Another great function call is describe(). This gives count, mean, std, min, and...
the two middle quartiles (think boxplots)

In [ ]:
raw.describe().round(1)

Another really interesting call is value_counts()

This call will print out whatever column you send to it with the counts
for all of the unieque values within it. 

So unlike above, you have the prevelance of each value, not just what each column contains



In [ ]:
raw['MSZoning'].value_counts().reset_index()

Very similar to what I like to do with a pivot table, but is so much quicker and can be done on multiple columns quite easily.

In [ ]:
for colu in raw:                             #grab column out columns from the dataset. Will cycle through them, but will reference each one as "colu" in turn
    if len(raw[f"{colu}"].unique())<15:   
      print(f'Column Name: {colu}')           # printing out the column name
      print(f"Unique Values: {raw[f'{colu}'].unique()}")    #printing out all the unieque values within raw for each column
      print(f"Value Counts for each variable: \n{raw[f'{colu}'].value_counts()}\n\n")

I can also filter columns much more easily than in excel, I can either omit them by using a data.drop(columns="what","ever","we","dont",want")
or by creating a list of the columns we do want to keep



In [ ]:
colers=['MSZoning', 'LotFrontage', 'LotArea',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'ExterQual', 'ExterCond', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'GarageCars',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'YrSold', 'SaleType', 'SalePrice']

In [ ]:
raw_small=raw[colers]

In [ ]:
raw_small

Let's create one of our favorite columns that we had created, tot_porch.
It is easy to do additions and create new columns: simply create the new column first, then insert the combination/calculations of columns that you want

In [ ]:
raw_small['tot_porch']=raw_small['EnclosedPorch']+raw_small['3SsnPorch']+raw_small['ScreenPorch'];
raw_small.drop(columns=['EnclosedPorch', '3SsnPorch','ScreenPorch'],inplace=True)

Now, to recreate the correlation chart that we created in excel, 
that allowed us to drop useless-ish columns before and focus on what mattered. 

In [ ]:
raw_small

In [ ]:
dfer = pd.DataFrame(part2_data[part2_data.columns])
corr = dfer.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

Great, now let's filter out all the columns that show a good correlation with our target "SalePrice"

In [ ]:
correlated=pd.DataFrame(index=part2_data.index)                                #create the empty space first
for colu in part2_data.drop(columns=['Id']):                                   #cycle through each column
  if abs(part2_data['SalePrice'].corr(part2_data[f'{colu}'])) >=.5:            # only get columns correlated with SalePrice .5 or higher
    print(colu)                                                                # Print and add to new dataframe "correlated"
    correlated[f'{colu}']=(part2_data[f'{colu}'])

In [ ]:
dfer = pd.DataFrame(correlated[correlated.columns])
corr = dfer.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

Look at the highy red areas, the quality ratings are very similar AND have high impact on the target. What one shall we pick to keep?

Also, I have BOTH YearBuilt and log_yfn (years from now). What one to choose now? 

In [ ]:
cor=correlated.drop(columns=['FullBath', 'TOT_rooms', 'YearBuilt', 'quality_mult', 'log_qual'])

The same visualizations that we created in Excel, are done here much more easily. 
There are a multitude of things we can do to visualize data, but I want to get to modeling and talk about that

In [ ]:
cor

Before we model, just want to show how much easier it is to edit and controll graphs in Python compared to Excel

In [ ]:
ax = sns.boxplot(x="Neighborhood", y="SalePrice", data=raw).set_title("Our Old Friend the Neighborhood Boxplot")
plt.xticks(rotation=90)

## OUR FIRST MODEL!

Let us define another metric that we will look at today. 
Root Mean Squared Error is a great statistic, and it punishes outliers very harshly.

In [ ]:
def rmse(predictions, targets):

    differences = predictions - targets                       #the DIFFERENCEs.

    differences_squared = differences ** 2                    #the SQUAREs of ^

    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^

    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^

    return rmse_val   

We need to define out train and test data. One we will use to train our model on (think homework)
the other test set (like a test with unseen answers) will be used for validation. 

In [ ]:
#part 2 data split
X_train, X_test, y_train, y_test = train_test_split(part2_data.drop(columns='SalePrice'), part2_data.SalePrice, test_size=0.33, random_state=42)

In [ ]:
#part 2 data split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(correlated.drop(columns='SalePrice'), correlated.SalePrice, test_size=0.33, random_state=42)

In [ ]:
# Raw splity, lets start here!
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(raw.drop(columns='SalePrice'), raw.SalePrice, test_size=0.33, random_state=42)

### This is how Pythjon and ScikitLearn work

Model choice comes here

fitting(features,target)   #Where the model trains on past data

predicting(train_features/test_features)    #the model then gets to test itself 

scoring(test_features,test_target)       

In [ ]:
reg = LinearRegression() #defining our model

#fitting the model
reg = reg.fit(X_train_raw,y_train_raw)

#Predicting both training and test sets so I can compare to reality
pred_train = reg.predict(X_train_raw)
pred_test = reg.predict(X_test_raw)

#Scoring both the training sets and the test sets
y_pred = reg.score(X_test_raw,y_test_raw)
ttree = reg.score(X_train_raw, y_train_raw)

#testing metrics are output here
print("Training score of Regression Model",ttree)
print("Testing Score of Regression Model",y_pred)
print(f'\nTraining RMSE of model: {rmse(pred_train,y_train_raw)}')
print(f'Testing RMSE of model: {rmse(pred_test,y_test_raw)}')

#### What happened here?? We did not check our data before we began modeling.

Right now our model is responding to catagorical values still in the raw dataset (MSZoning here).

We also need to check for NaN values (empty), values that are extreme (think infinite), and any other errors we want!

Excel is not always good at telling us where or when we have missing values, sometimes in numerical columns it will simply just add a 0!

In [ ]:
raw_data_na = (raw.isnull().sum() / len(raw)) * 100 #creating percentages of missing values (missing/total * 100)
raw_data_na = raw_data_na.drop(raw_data_na[raw_data_na == 0].index).sort_values(ascending=False)[:40] #Focus
missing_data_raw = pd.DataFrame({'Missing Ratio in Raw' :raw_data_na})
missing_data_raw.head(20)

#### To test our our Excel-based feature cleaning and engineering, lets see if we can build a model!

In [ ]:
data_na = (part2_data.isnull().sum() / len(raw)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:40]
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data

In [ ]:
correlated[correlated.YearBuilt.isna()]

In [ ]:
data_na = 100-(np.isfinite(part2_data).sum() / len(raw)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:40]
missing_data = pd.DataFrame({'Infinate Ratio' :data_na})
missing_data

### Lets give it a shot with our cleaned dataset!

In [ ]:
#fitting the model we hav already defined
reg = reg.fit(X_train,y_train)

#Predicting both training and test sets so I can compare to reality
pred_train = reg.predict(X_train)
pred_test = reg.predict(X_test)

#Scoring both the training sets and the test sets
y_pred = reg.score(X_test,y_test)
ttree = reg.score(X_train, y_train)

#testing metrics are output here
#default score for regression model is R^2
print("Training score of part2 Regression Model",ttree)
print("Testing Score of part2 Regression Model",y_pred)

#Mean Absolute Error is a great metric, it is the average deviation between prediction and reality
print(f'\nTraining MAE of part2 model: {mae(pred_train,y_train)}')
print(f'Testing MAE of part2 model: {mae(pred_test,y_test)}')

#RMSE - Root Mean Squared error, another typical metic, this one has a squared term to penalize big misses
print(f'\nTraining RMSE of part2 model: {rmse(pred_train,y_train)}')
print(f'Testing RMSE of part2 model: {rmse(pred_test,y_test)}')

In [ ]:
#With our much smaller dataset of 17 features
reg2 = LinearRegression() #redefining our model
reg2 = reg.fit(X_train_s,y_train_s)
y_pred_s = reg2.score(X_test_s,y_test_s)
ttree_s = reg2.score(X_train_s, y_train_s)
pred_train_s = reg.predict(X_train_s)
pred_test_s = reg.predict(X_test_s)
print("Training score of small Regression Model",ttree_s)
print("Testing Score of small Regression Model",y_pred_s)
print(f'\nTraining RMSE of small model: {rmse(pred_train_s,y_train_s)}')
print(f'Testing RMSE of small model: {rmse(pred_test_s,y_test_s)}')
print(f'\nTraining MAE of small: {mae(pred_train_s,y_train_s)}')
print(f'Testing MAE of small model: {mae(pred_test_s,y_test_s)}')

WOW! We explained 84% of the variation in the dataset! But we still have a long way to go.

We are going to revisit this, for if we can explain the same amount of variation in the data with a much smaller model,
we can more easily deploy it, explain it, and make it faster!

# Here were my first thoughts of transforming the Raw dataset:

### Let's take that first model as a baseline, if we can't beat that easily, maybe we stop there!
 

In [ ]:
#very robust model, able to handle outliers, typically a great place to start
forest = RandomForestRegressor()

#fitting the model to the data
forest = forest.fit(X_train,y_train)
#Score the results
train_score_forest = forest.score(X_train, y_train)
test_score_forest = forest.score(X_test,y_test)

pred_train_forest = forest.predict(X_train)
pred_test_forest = forest.predict(X_test)
print("Training score of RF Model",train_score_forest)
print("Testing Score of RF Model",test_score_forest)
print(f'\nTraining RMSE of RF model: {rmse(pred_train_forest,y_train)}')
print(f'Testing RMSE of RF model: {rmse(pred_test_forest,y_test)}')
print(f'\nTraining MAE of RF model: {mae(pred_train_forest,y_train)}')
print(f'Testing MAE of RF model: {mae(pred_test_forest,y_test)}')

Let's not immediately start throwing models at the data, for although we have improved, we have not done much other than throwing fancy algos at the data.

This will be our new baseline, and we need to work hard to beat this!

### Starting with the RAW dataset, what do we need to fix or what can we improve?

Maybe let's look for outliers, and there are many ways to do this with statistics, so I will not bore you here with them.

Revisiting week 2 leaves us with this one outler set, how do we visualize and remove the value in python?

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = raw['GrLivArea'], y = raw['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
train = raw.drop(raw[(raw['GrLivArea']>4000) & (raw['SalePrice']<300000)].index) 
#very similar to an excel foruma here, but in the same "new_thing = data[filter1]" notation

In [ ]:
full_data_clean=train.copy()

There are many ways to clean data, and to get python code to do what you want. 
To fill empty values, the .fillna() call is your best friend.
You can fill with a constant, means, or medians.

In [ ]:
#All of these missing values refer to certain columns where features are not present. 
#Missing means 0 here. 
full_data_clean["PoolQC"] = full_data_clean["PoolQC"].fillna(0)
full_data_clean["MiscFeature"] = full_data_clean["MiscFeature"].fillna(0)
full_data_clean["Alley"] = full_data_clean["Alley"].fillna(0)
full_data_clean["Fence"] = full_data_clean["Fence"].fillna(0)
full_data_clean["FireplaceQu"] = full_data_clean["FireplaceQu"].fillna(0)
full_data_clean["MasVnrArea"] = full_data_clean["MasVnrArea"].fillna(0)
full_data_clean["Electrical"] = full_data_clean["Electrical"].fillna(0)
full_data_clean["LotFrontage"] = full_data_clean["LotFrontage"].fillna(0)

If you have large segments of data, you can also use a loop to move through all the columns you want.
Below are all the basement and garage metrics (both where NA means 0)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 
            'BsmtFullBath', 'BsmtHalfBath','GarageYrBlt', 'GarageArea', 'GarageCars'):
    full_data_clean[col] = full_data_clean[col].fillna(0)

In [ ]:
#These variables are catagorical, so we can fill them with a different label
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    full_data_clean[col] = full_data_clean[col].fillna('None')

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    full_data_clean[col] = full_data_clean[col].fillna("None")

In [ ]:
full_data_clean["MasVnrType"] = full_data_clean["MasVnrType"].fillna("None")

But What about values that have no difinitive zero value in the data?

MSZoning and Electrical should all have values but do not.


This is more similar to a problem we see in the real world, most of Baxter data is in this shape,

and what we choose to do here can have large influences on the outcome of any model

Lets look at MSZoning

In [ ]:
full_data_clean['MSZoning'].value_counts()

In [ ]:
full_data_clean['MSZoning'] = full_data_clean['MSZoning'].fillna('RL') #Filling with Meaian data, easily done

In [ ]:
full_data_clean['Electrical'].value_counts()

In [ ]:
full_data_clean['Electrical'] = full_data_clean['Electrical'].fillna('SBrkr') #same here too

Are there other fancy options? 

Yes, but for the percentage of missing data here (small), and the impact it will have being somewhat simplified (small)

We can take the easy route here. 

What about all of the columns that are not as easily handled?
I have spoken before about dummy and one-hot encoding. Lets look really quicky at Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(full_data_clean[c].values)) 
    full_data_clean[c] = lbl.transform(list(full_data_clean[c].values))

# shape        
print('Shape all_data: {}'.format(full_data_clean.shape))

## There are Many Ways to Clean this Dataset:
### I chose to code a bit more, but be able to repeatably create good data down the line

### We want calculations that can handle problems and mistakes better, I have accounted for misspelling all below

In [ ]:
#we can look at features that are non-numerical first
train.select_dtypes(include=['object'])

In [ ]:
#fill the columns with zeros or nones that are easy
train['GarageYrBlt']=train['GarageYrBlt'].fillna(0)
train['MasVnrArea']=train['MasVnrArea'].fillna(0)
train['LotFrontage']=train['LotFrontage'].fillna(0)
train['MasVnrType']=train['MasVnrType'].fillna("None")

In [ ]:
#The APPLY function is super fast in python, therefore these calls go through the dataset row by row but much faster than a loop would
train['Alley']=train['Alley'].apply(lambda x: 1 if x!=0 else 0)
train['LotShape']=train['LotShape'].apply(lambda x: 1 if x=='Reg' or x=='IR1' else 0)
train['LandContour']=train['LandContour'].apply(lambda x: 1 if x=='Lvl' else 0)
train['Utilities']=train['Utilities'].apply(lambda x: 1 if x=='AllPub' or x=='NoSewr' else 0)
train['LotConfig']=train['LotConfig'].apply(lambda x: 1 if x=='Inside' or x=='CulDSac' else 0)
train['LotConfig']=train['LotConfig'].apply(lambda x: 1 if x=='Inside' or x=='CulDSac' else 0)
train['LandSlope']=train['LandSlope'].apply(lambda x: 1 if x=='Gtl' else 0)
train['ExterQual']=train['ExterQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
train['ExterCond']=train['ExterCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
train['BsmtQual']=train['BsmtQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['BsmtCond']=train['BsmtCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['BsmtExposure']=train['BsmtExposure'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['BsmtFinType1']=train['BsmtFinType1'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
train['BsmtFinType2']=train['BsmtFinType2'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
train['HeatingQC']=train['HeatingQC'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['CentralAir']=train['CentralAir'].apply(lambda x: 1 if x=='Y' else 0)
train['KitchenQual']=train['KitchenQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
train['Functional']=train['Functional'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
train['FireplaceQu']=train['FireplaceQu'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['GarageFinish']=train['GarageFinish'].apply(lambda x: 3 if x=='Fin' else (2 if x=='RFn' else (1 if x=='Unf' else 0)))
train['GarageQual']=train['GarageQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['GarageCond']=train['GarageCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['PavedDrive']=train['PavedDrive'].apply(lambda x: 1 if x=='Y' or x=='P' else 0)
train['PoolQC']=train['PoolQC'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
train['Fence']=train['Fence'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))

#we can even create new features here
train['TOT_bath']=train['FullBath']+(.5*train['HalfBath'])

In [ ]:
# Creating One-Hot-Encodings for catagorical columns
# Then deleting the columns in the next row using inplace

train=pd.concat([train,pd.get_dummies(train.MSZoning)],axis=1)
train.drop(columns=('MSZoning'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.MSSubClass)],axis=1)
train.drop(columns=('MSSubClass'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.Neighborhood)],axis=1)
train.drop(columns=('Neighborhood'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.BldgType)],axis=1)
train.drop(columns=('BldgType'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.HouseStyle)],axis=1)
train.drop(columns=('HouseStyle'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.GarageType)],axis=1)
train.drop(columns=('GarageType'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.SaleCondition)],axis=1)
train.drop(columns=('SaleCondition'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.SaleType)],axis=1)
train.drop(columns=('SaleType'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.Condition1)],axis=1)
train.drop(columns=('Condition1'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.MasVnrType)],axis=1)
train.drop(columns=('MasVnrType'),inplace=True)

train=pd.concat([train,pd.get_dummies(train.Foundation)],axis=1)
train.drop(columns=('Foundation'),inplace=True)

In [ ]:
train.reset_index(inplace=True)
train=train.drop(columns=['Id','Street','Condition2','RoofStyle','RoofMatl','Exterior1st','Exterior2nd', 'Heating',
                          'Electrical','MiscFeature'])

How did we do??

In [ ]:
train.select_dtypes(include=['object']).head(5)

In [ ]:
#Lets make sure we don't have any Extreme or missing values 
for column in train:
    print(f'for column {column}') 
    print(f'\tMAX={train[column].max()}') 
    print(f'\tMIN={train[column].min()}') 
    print(f'\tNAN={train[column].isna().sum()}')

### Functions:
functions are small parts of code that you can define to do things. 
They work similarly to a lot of what has been done earlier ("Iwanttodosomethingto(thisDF)")
You create the function, then tell it what to expect incoming, 
you do things within it, then output (or not) what you want!

In [ ]:
def year_transform(df, years_column):             #defining incoming dataframe and column for transformation
    for column in years_column:                   #loop
        df[column]=df[column]-(df[column].min()-1) #want numbers 1-> whatever with oldest houses being a large number
        df[column]=df[column].apply(lambda x: 0 if x==0 else np.log10(x)) #we know we need to take the Log for distrobution problems
    return df

In [ ]:
#now we can quickly and easlily send lots of columns in, and get them all traqnsformed in one line!
years_column=['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
train=year_transform(train,years_column)

## Lets get to more modeling!

In [ ]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import RidgeCV, LassoLars, Ridge, ElasticNet, HuberRegressor
from sklearn.preprocessing import QuantileTransformer, quantile_transform

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(train.drop(columns='SalePrice'), train.SalePrice, test_size=0.33, random_state=42)

In [ ]:
log = LinearRegression()
log = log.fit(X_train3,y_train3)
y_pred3 = log.score(X_test3,y_test3)
ttree3 = log.score(X_train3, y_train3)
pred3_train = log.predict(X_train3)
pred3_test = log.predict(X_test3)
print("Training score of Tree Model",ttree3)
print("Testing Score of Tree Model",y_pred3)
print(f'\nTraining RMSE of model: {rmse(pred3_train,y_train3)}')
print(f'Testing RMSE of model: {rmse(pred3_test,y_test3)}')

Looks like we are already doing better!
Much closer to out baseline preformer, did our changes effect the baseline as well?

In [ ]:
tree = RandomForestRegressor()
tree = tree.fit(X_train3,y_train3)
y_pred3 = tree.score(X_test3,y_test3)
ttree3 = tree.score(X_train3, y_train3)
pred3_trainRF = tree.predict(X_train3)
pred3_testRF = tree.predict(X_test3)
print("Training score of Tree Model",ttree3)
print("Testing Score of Tree Model",y_pred3)
print(f'\nTraining RMSE of model: {rmse(pred3_trainRF,y_train3)}')
print(f'Testing RMSE of model: {rmse(pred3_testRF,y_test3)}')

Yes, up to 90% explained variance. Very cool!
What can we do to inspect what is actually going on point-by-point, or how can we improve the linear model

### Residual Test

What is this "residuals" stuff? This is a basic test that attempts to uncover if we have left information out of consideration of the model. 

How? The residuals are the measurements between the predicted scores and the actual scores.

If there is a pattern left in those differences, there is a trend missing from our predictions. 

In [ ]:
from sklearn.metrics import median_absolute_error, r2_score
f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(10, 12))

ax0[0].scatter(pred3_test, y_test3, s=8)
ax0[0].plot([0, 7e5], [0, 7e5], '--k')
ax0[0].set_ylabel('True target')
ax0[0].set_xlabel('Predicted target')
ax0[0].text(s='Linear Regression', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test3, pred3_test), median_absolute_error(y_test3, pred3_test)))
ax0[0].set_xlim([0, 7e5])
ax0[0].set_ylim([0, 7e5])
ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[0].scatter(pred3_test, (pred3_testRF - y_test3), s=8)
ax1[0].set_ylabel('Residual')
ax1[0].set_xlabel('Predicted target')
ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax0[1].scatter(pred_test, y_test3, s=8)
ax0[1].plot([0, 7e5], [0, 7e5], '--k')
ax0[1].set_ylabel('True target')
ax0[1].set_xlabel('Predicted target')
ax0[1].text(s='Random Forest Regression', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test3, pred3_testRF), median_absolute_error(y_test3, pred3_testRF)))
ax0[1].set_xlim([0, 7e5])
ax0[1].set_ylim([0, 7e5])
ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[1].scatter(pred3_testRF, (pred3_testRF - y_test3), s=8)
ax1[1].set_ylabel('Residual')
ax1[1].set_xlabel('Predicted target')
ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

f.suptitle("Ames housing data: selling price", y=0.035)

plt.show()

Both look pretty good (as expected) We do see a little droop in the right hand side of the graphs.

This means that for the more extreme prices, we are underfitting. What does that remind us on from DDD part_2?

TRANSFORMS!!!

Something I like to do once I get into heavy duty stuff, is seporate the target from the variables, therefore it is untouched by any transformations, 
as well as being easily acessed for graphs and such

In [ ]:
target=train.pop('SalePrice')

In [ ]:
train.SalePrice  #WILL FAIL

In [ ]:
sns.distplot(target, fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(target)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(target, plot=plt)
plt.show()

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
target_log = np.log1p(target)

#Check the new distribution 
sns.distplot(target_log, fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(target_log)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(target_log, plot=plt)
plt.show()

As you can see, our distrobution is much more normal and the Q-Q Plot (the bottom graphs) show how far a distrobution is from normal 
by fitting it to a straight line. Looks like we are much more on target now. 

### But will that actually help predictions? 

back_from_log = np.expm1(target_log)

In [ ]:
X_trainL, X_testL, y_trainL, y_testL = train_test_split(train, target_log, test_size=0.33, random_state=42)

In [ ]:
log3 = LinearRegression()
log3 = log3.fit(X_trainL,y_trainL)
y_predL = log3.score(X_testL,y_testL)
ttreeL = log3.score(X_trainL, y_trainL)
predL = log3.predict(X_trainL)
predNL = np.expm1(predL)
predL_test = log3.predict(X_testL)
predNL_test = np.expm1(predL_test)

print("Training score of Reg Model",ttreeL)
print("Testing Score of Reg Model",y_predL)
print(f'\nTraining RMSE of Reg model: {rmse(predNL,y_trainL)}')
print(f'Testing RMSE of Reg model: {rmse(predNL_test,y_testL)}')
print(f'\nTraining MAE of Reg model: {mae(predNL,y_trainL)}')
print(f'Testing MAE of Reg model: {mae(predNL_test,y_testL)}')

In [ ]:
treeL = RandomForestRegressor()
treeL = treeL.fit(X_trainL,y_trainL)
y_predLF = treeL.score(X_testL,y_testL)
ttreeLF = treeL.score(X_trainL, y_trainL)
predLF = treeL.predict(X_trainL)
predNLF = np.expm1(predLF)
predL_testF = treeL.predict(X_testL)
predNL_testF = np.expm1(predL_testF)


print("Training score of Forest Model",ttreeLF)
print("Testing Score of Forest Model",y_predLF)
print(f'\nTraining RMSE of Forest model: {rmse(predNLF,y_trainL)}')
print(f'Testing RMSE of Forest model: {rmse(predNL_testF,y_testL)}')
print(f'\nTraining MAE of Forest model: {mae(predNLF,y_trainL)}')
print(f'Testing MAE of Forest model: {mae(predNL_testF,y_testL)}')

We have basically flipped the results on their head. Our transformation has hurt the RF model, and has helped the linear model.
We always have to make decisions in our DS projects that will have large impacts on our outputs

In [ ]:
tester = np.expm1(y_testL)

In [ ]:
from sklearn.metrics import median_absolute_error, r2_score
f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(6.5, 8))

ax0[0].scatter(predNL_test, tester, s=8)
ax0[0].plot([0, 7e5], [0, 7e5], '--k')
ax0[0].set_ylabel('True target')
ax0[0].set_xlabel('Predicted target')
ax0[0].text(s='Linear Regression', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(tester, predNL_test), median_absolute_error(tester, predNL_test)))
ax0[0].set_xlim([0, 7e5])
ax0[0].set_ylim([0, 7e5])
ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[0].scatter(predNL_test, (predNL_test - tester), s=8)
ax1[0].set_ylabel('Residual')
ax1[0].set_xlabel('Predicted target')
ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax0[1].scatter(predNL_testF, tester, s=8)
ax0[1].plot([0, 7e5], [0, 7e5], '--k')
ax0[1].set_ylabel('True target')
ax0[1].set_xlabel('Predicted target')
ax0[1].text(s='Fandom Forest Regression', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(tester, predNL_testF), median_absolute_error(tester, predNL_testF)))
ax0[1].set_xlim([0, 7e5])
ax0[1].set_ylim([0, 7e5])
ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[1].scatter(predNL_testF, (predNL_testF - tester), s=8)
ax1[1].set_ylabel('Residual')
ax1[1].set_xlabel('Predicted target')
ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

f.suptitle("Ames housing data: selling price", y=0.035)

plt.show()

LOOK! our residuals are streghtened, and our R^2 is up!

We are much more toward the path that we want, even beating our RF algorithm.

## Model Simplification

Lets see if we can make a better and simpler model in a different way.

We tried only selecting correlated values, and did pretty well, are there other options.

We have three major shoices here:

1.) Univariate/Filter based Selection -- Choosing features ourselves by how they impact the model in some way (correlation)

2.) Wrapper Methods -- Uses one single ML algorithm and uses its performance as evaluation criteria, 
                        we chooses features based on their influence on preformance. (p-value of feature importance)

3.) Embedded Methods -- Repeadedly tries different combinations of features to achieve the best feature set. 

We have tried #1 before without too much success,
So lets build a model with #2, we are going to take out the pvalues of each predictive column,
then filter for ones that have impact on the output. 

In [ ]:
selected_columns = X_trainL.columns
import statsmodels.api as sm
def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns

SL = 0.02
data_modeled, selected_columns = backwardElimination(X_trainL.iloc[:,1:].values, y_trainL.values, SL, selected_columns)

In [ ]:
selected_columns

In [ ]:
X_train_s = X_trainL[selected_columns]
X_test_s = X_testL[selected_columns]
y_train_s = y_trainL
y_test_s = y_testL


In [ ]:
#fitting the model to the data
reg_s = reg.fit(X_train_s,y_train_s)
#Score the results
reg = reg.fit(X_train_s,y_train_s)
y_pred = reg.score(X_test_s,y_test_s)
ttree = reg.score(X_train_s, y_train_s)
pred_train = reg.predict(X_train_s)
pred = np.expm1(pred_train)
pred_test = reg.predict(X_test_s)
pret = np.expm1(pred_test)
print("Training score of Regression Model",ttree)
print("Testing Score of Regression Model",y_pred)
print(f'\nTraining RMSE of model: {rmse(pred,y_train_s)}')
print(f'Testing RMSE of model: {rmse(pret,y_test_s)}')

In [ ]:
#just for comparisons sake this is the old model with lots of features
log3 = LinearRegression()
log3 = log3.fit(X_trainL,y_trainL)
y_predL = log3.score(X_testL,y_testL)
ttreeL = log3.score(X_trainL, y_trainL)
predL = log3.predict(X_trainL)
predNL = np.expm1(predL)
predL_test = log3.predict(X_testL)
predNL_test = np.expm1(predL_test)

print("Training score of Reg Model",ttreeL)
print("Testing Score of Reg Model",y_predL)
print(f'\nTraining RMSE of Reg model: {rmse(predNL,y_trainL)}')
print(f'Testing RMSE of Reg model: {rmse(predNL_test,y_testL)}')
print(f'\nTraining MAE of Reg model: {mae(predNL,y_trainL)}')
print(f'Testing MAE of Reg model: {mae(predNL_test,y_testL)}')

In [ ]:
X_trainL.shape

In [ ]:
X_train_s.shape

Thinking about model size is always important, 
optimizing to a smaller model usually means more consistant results (less testing error)
as well as quicker predictions.

Looks like there is a good way to optimize this model fo speed as well!

## Fine Tuning the Model Hyperperameters

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
models = { 
    'TTR': TransformedTargetRegressor()}

params = {
    'TTR': {
        'regressor':  [ 
            LassoLars(alpha=0.001),
            LassoLars(alpha=0.01),
            LassoLars(alpha=0.1),
            LassoLars(alpha=1),
            LassoLars(alpha=10),
            Ridge(alpha=1),
            Ridge(alpha=6),
            Ridge(alpha=8),
            Ridge(alpha=10),
            Ridge(alpha=12),
            Ridge(alpha=30),
            ElasticNet(alpha=0.001),
            ElasticNet(alpha=0.01),
            ElasticNet(alpha=0.1),
            ElasticNet(alpha=1),
            ElasticNet(alpha=10),
            ElasticNet(alpha=100),
            HuberRegressor(alpha=0.001),
            HuberRegressor(alpha=0.01),
            HuberRegressor(alpha=0.1),
            HuberRegressor(alpha=1),
            HuberRegressor(alpha=10),
            HuberRegressor(alpha=100)],
        'transformer': [QuantileTransformer(n_quantiles=750,output_distribution='normal'),
                       QuantileTransformer(n_quantiles=300,output_distribution='normal')]},
        'regressor__alpha': [1,10]
}

In [ ]:
class EstimatorSelectionHelper:
    
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, **grid_kwargs):
        for key in self.keys:
            print('Running GridSearchCV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            grid_search = GridSearchCV(model, params, cv=5, **grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
        print('Done.')
    
    def score_summary(self, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [ ]:
helper1 = EstimatorSelectionHelper(models, params)
helper1.fit(X_train_s, y_train_s, scoring='r2', n_jobs=1)
helper1.score_summary().head(20)

In [ ]:
helper1 = EstimatorSelectionHelper(models, params)
helper1.fit(X_train3, y_train3, scoring='r2', n_jobs=1)
helper1.score_summary().head(20)

After repeated cross validation, our model selection tool shows us two things, one, that Ridge() trasformer is better than our standard LinearRegression,
and also, that having more features "in this case" helped.
Probably why: Ridge Regression actually attempts to limit the number of varables that are allowed to contribute to the model. 
Therefore, it is limiting the features interally and scoring off that.

In [ ]:
helper1.score_summary().params[0]

In [ ]:
regr_trans3 = TransformedTargetRegressor(
    regressor= ElasticNet(alpha=0.001),
    transformer=QuantileTransformer(n_quantiles=750,
                                    output_distribution='normal'))

In [ ]:
regr_trans = TransformedTargetRegressor(
    regressor= Ridge(alpha=8),
    transformer=QuantileTransformer(n_quantiles=750,
                                    output_distribution='normal'))

In [ ]:
regr_trans3 = regr_trans3.fit(X_train3,y_train3)
y_pred = regr_trans3.score(X_test3,y_test3)
ttree = regr_trans3.score(X_train3, y_train3)
pred_train = regr_trans3.predict(X_train3)
pred_test = regr_trans3.predict(X_test3)
print("Training score of Tree Model",y_pred)
print("Testing Score of Tree Model",ttree)
print(f'\nTraining RMSE of model: {rmse(pred_train,y_train3)}')
print(f'Testing RMSE of model: {rmse(pred_test,y_test3)}')
print(f'\nTraining MAE of Reg model: {mae(pred_train,y_train3)}')
print(f'Testing MAE of Reg model: {mae(pred_test,y_test3)}')

In [ ]:
regr_trans = regr_trans.fit(X_train3,y_train3)
y_pred = regr_trans.score(X_test3,y_test3)
ttree = regr_trans.score(X_train3, y_train3)
pred_train = regr_trans.predict(X_train3)
pred_test = regr_trans.predict(X_test3)
print("Training score of Tree Model",y_pred)
print("Testing Score of Tree Model",ttree)
print(f'\nTraining RMSE of model: {rmse(pred_train,y_train3)}')
print(f'Testing RMSE of model: {rmse(pred_test,y_test3)}')
print(f'\nTraining MAE of Reg model: {mae(pred_train,y_train3)}')
print(f'Testing MAE of Reg model: {mae(pred_test,y_test3)}')

So awesome! Let's save this for later!

In [ ]:
import pickle
pickle.dump(regr_trans, open('FinalModel.sav', 'wb'))

In [ ]:
with open('FinalModel.sav', "rb") as f:
        SavedModel = pickle.load(f)
did_I_save=SavedModel

In [ ]:
did_I_save

In [ ]:
def fil_zero(df):
    df['GarageYrBlt']=df['GarageYrBlt'].fillna(0)
    df['MasVnrArea']=df['MasVnrArea'].fillna(0)
    df['LotFrontage']=df['LotFrontage'].fillna(0)
    df['MasVnrType']=df['MasVnrType'].fillna("None")
    return df

def fix_cat_feat(df):
    df['Alley']=df['Alley'].apply(lambda x: 1 if x!=0 else 0)
    df['LotShape']=df['LotShape'].apply(lambda x: 1 if x=='Reg' or x=='IR1' else 0)
    df['LandContour']=df['LandContour'].apply(lambda x: 1 if x=='Lvl' else 0)
    df['Utilities']=df['Utilities'].apply(lambda x: 1 if x=='AllPub' or x=='NoSewr' else 0)
    df['LotConfig']=df['LotConfig'].apply(lambda x: 1 if x=='Inside' or x=='CulDSac' else 0)
    df['LotConfig']=df['LotConfig'].apply(lambda x: 1 if x=='Inside' or x=='CulDSac' else 0)

    df['LandSlope']=df['LandSlope'].apply(lambda x: 1 if x=='Gtl' else 0)
    df['ExterQual']=df['ExterQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
    df['ExterCond']=df['ExterCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
    df['BsmtQual']=df['BsmtQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['BsmtCond']=df['BsmtCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['BsmtExposure']=df['BsmtExposure'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['BsmtFinType1']=df['BsmtFinType1'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
    df['BsmtFinType2']=df['BsmtFinType2'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
    df['HeatingQC']=df['HeatingQC'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['CentralAir']=df['CentralAir'].apply(lambda x: 1 if x=='Y' else 0)
    df['KitchenQual']=df['KitchenQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else 1))))
    df['Functional']=df['Functional'].apply(lambda x: 5 if x=='GLQ' else (4 if x=='ALQ' else (3 if x=='BLQ' else (2 if x=='Rec' else (1 if x=='LwQ' else 0)))))
    df['TOT_bath']=df['FullBath']+(.5*df['HalfBath'])
    df['FireplaceQu']=df['FireplaceQu'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['GarageFinish']=df['GarageFinish'].apply(lambda x: 3 if x=='Fin' else (2 if x=='RFn' else (1 if x=='Unf' else 0)))
    df['GarageQual']=df['GarageQual'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['GarageCond']=df['GarageCond'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['PavedDrive']=df['PavedDrive'].apply(lambda x: 1 if x=='Y' or x=='P' else 0)
    df['PoolQC']=df['PoolQC'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    df['Fence']=df['Fence'].apply(lambda x: 5 if x=='Ex' else (4 if x=='Gd' else (3 if x=='TA' else (2 if x=='Fa' else (1 if x=='Po' else 0)))))
    return df

def fix_dummies(df):
    df=pd.concat([df,pd.get_dummies(df.MSZoning)],axis=1)
    df.drop(columns=('MSZoning'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.MSSubClass)],axis=1)
    df.drop(columns=('MSSubClass'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.Neighborhood)],axis=1)
    df.drop(columns=('Neighborhood'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.BldgType)],axis=1)
    df.drop(columns=('BldgType'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.HouseStyle)],axis=1)
    df.drop(columns=('HouseStyle'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.GarageType)],axis=1)
    df.drop(columns=('GarageType'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.SaleCondition)],axis=1)
    df.drop(columns=('SaleCondition'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.SaleType)],axis=1)
    df.drop(columns=('SaleType'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.Condition1)],axis=1)
    df.drop(columns=('Condition1'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.MasVnrType)],axis=1)
    df.drop(columns=('MasVnrType'),inplace=True)

    df=pd.concat([df,pd.get_dummies(df.Foundation)],axis=1)
    df.drop(columns=('Foundation'),inplace=True)

    df.reset_index(inplace=True)
    df=df.drop(columns=['Id','Street','Condition2','RoofStyle','RoofMatl','Exterior1st','Exterior2nd', 'Heating',
                              'Electrical','MiscFeature'])
    return df

def years_transform(df):
    for column in ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']:
        df[column]=df[column]-(df[column].min()-1)
        df[column]=df[column].apply(lambda x: 0 if x==0 else np.log10(x))
    return df

In [ ]:
url = 'https://raw.githubusercontent.com/ChristianRLynn/DataSets/main/train.csv'
raw = pd.read_csv(url, error_bad_lines=False); raw.drop(columns='SalePrice',inplace=True)

first = fil_zero(raw)

second = fix_cat_feat(first)

third = fix_dummies(second)

fourth = years_transform(third)

In [ ]:
with open('FinalModel.sav', "rb") as f:
        SavedModel = pickle.load(f)
did_I_save=SavedModel

In [ ]:
did_I_save.predict(third)[1:500]

# That's All Folks! We have a model saved!
## Trughfully there is a lot to discuss about creating model pipelines and model deployment,
## but, so little time, so much fun left. 
THANKS!

# Extra FUN

In [ ]:
log_col_list=['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
              'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF',
              'GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF',
              'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']
train2=log_trans(train,log_col_list)

In [ ]:
years_column=['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
train2=year_transform(train2,years_column)

In [ ]:
train2.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
train3=scaler.fit_transform(train2)
train3=pd.DataFrame(data=train3,    # values
              index=train2.index,    # 1st column as index
              columns=train2.columns)

In [ ]:
train3.shape

In [ ]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(train3, target, test_size=0.33, random_state=42)

In [ ]:
X_train4l, X_test4l, y_train4l, y_test4l = train_test_split(train3, target_log, test_size=0.33, random_state=42)

In [ ]:
tree = RandomForestRegressor()
tree = tree.fit(X_train4,y_train4)
y_pred4 = tree.score(X_test4,y_test4)
ttree4 = tree.score(X_train4, y_train4)
pred4_train = tree.predict(X_train4)
pred4_test = tree.predict(X_test4)
print("Training score of Tree Model",ttree4)
print("Testing Score of Tree Model",y_pred4)
print(f'\nTraining RMSE of model: {rmse(pred4_train,y_train4)}')
print(f'Testing RMSE of model: {rmse(pred4_test,y_test4)}')

In [ ]:
treel = RandomForestRegressor()
treel = treel.fit(X_train4l,y_train4l)
y_pred4l = treel.score(X_test4l,y_test4l)
ttree4l = treel.score(X_train4l, y_train4l)
pred4_trainl = tree.predict(X_train4l)
pred4_testl = tree.predict(X_test4l)
print("Training score of Tree Model",ttree4)
print("Testing Score of Tree Model",y_pred4)
print(f'\nTraining RMSE of model: {rmse(pred4_trainl,y_train4l)}')
print(f'Testing RMSE of model: {rmse(pred4_testl,y_test4l)}')

In [ ]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import QuantileTransformer, quantile_transform

f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(6.5, 8))

regr = RidgeCV()
regr.fit(X_train4, y_train4)
y_pred = regr.predict(X_test4)

ax0[0].scatter(y_pred, y_test4, s=8)
ax0[0].plot([0, 7e5], [0, 7e5], '--k')
ax0[0].set_ylabel('True target')
ax0[0].set_xlabel('Predicted target')
ax0[0].text(s='Ridge regression \n without target transformation', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test4, y_pred), median_absolute_error(y_test4, y_pred)))
ax0[0].set_xlim([0, 7e5])
ax0[0].set_ylim([0, 7e5])
ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[0].scatter(y_pred, (y_pred - y_test4), s=8)
ax1[0].set_ylabel('Residual')
ax1[0].set_xlabel('Predicted target')
ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

regr_trans = TransformedTargetRegressor(
    regressor=RidgeCV(),
    transformer=QuantileTransformer(n_quantiles=900,
                                    output_distribution='normal'))
regr_trans.fit(X_train4, y_train4)
y_pred = regr_trans.predict(X_test4)

ax0[1].scatter(y_pred, y_test4, s=8)
ax0[1].plot([0, 7e5], [0, 7e5], '--k')
ax0[1].set_ylabel('True target')
ax0[1].set_xlabel('Predicted target')
ax0[1].text(s='Ridge regression \n with target transformation', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test4, y_pred), median_absolute_error(y_test4, y_pred)))
ax0[1].set_xlim([0, 7e5])
ax0[1].set_ylim([0, 7e5])
ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[1].scatter(y_pred, (y_pred - y_test4), s=8)
ax1[1].set_ylabel('Residual')
ax1[1].set_xlabel('Predicted target')
ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

f.suptitle("Ames housing data: selling price", y=0.035)

plt.show()

In [ ]:
f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(6.5, 8))

regr = TransformedTargetRegressor(
    regressor=RidgeCV(),
    transformer=QuantileTransformer(n_quantiles=900,
                                    output_distribution='normal'))
regr.fit(X_train4, y_train4)
y_pred = regr.predict(X_test4)

ax0[0].scatter(y_pred, y_test4, s=8)
ax0[0].plot([0, 7e5], [0, 7e5], '--k')
ax0[0].set_ylabel('True target')
ax0[0].set_xlabel('Predicted target')
ax0[0].text(s='Ridge regression \n with target transformation', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test4, y_pred), median_absolute_error(y_test4, y_pred)))
ax0[0].set_xlim([0, 7e5])
ax0[0].set_ylim([0, 7e5])
ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[0].scatter(y_pred, (y_pred - y_test4), s=8)
ax1[0].set_ylabel('Residual')
ax1[0].set_xlabel('Predicted target')
ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

regr_trans = TransformedTargetRegressor(
    regressor= RandomForestRegressor(bootstrap='False',criterion='mae',max_depth=12,max_features=20,min_samples_split=2,n_estimators= 200),
    transformer=QuantileTransformer(n_quantiles=900,
                                    output_distribution='normal'))
regr_trans.fit(X_train4, y_train4)
y_pred = regr_trans.predict(X_test4)

ax0[1].scatter(y_pred, y_test4, s=8)
ax0[1].plot([0, 7e5], [0, 7e5], '--k')
ax0[1].set_ylabel('True target')
ax0[1].set_xlabel('Predicted target')
ax0[1].text(s='RF regression \n with target transformation', x=-5e4,
            y=8e5, fontsize=12, multialignment='center')
ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
    r2_score(y_test4, y_pred), median_absolute_error(y_test4, y_pred)))
ax0[1].set_xlim([0, 7e5])
ax0[1].set_ylim([0, 7e5])
ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

ax1[1].scatter(y_pred, (y_pred - y_test4), s=8)
ax1[1].set_ylabel('Residual')
ax1[1].set_xlabel('Predicted target')
ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

f.suptitle("Ames housing data: selling price", y=0.035)

plt.show()

### Does Feature Engineering actually do much??

In [ ]:
def feat_engineering(df):
    df["porch_tot"]=df['EnclosedPorch']+df['3SsnPorch']+df['ScreenPorch']+df['WoodDeckSF']+df['OpenPorchSF']
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['base_bath_tot'] = df['BsmtFullBath'] + (df['BsmtFullBath']*.5)
    df['rooms_tot'] = df['TotalBsmtSF']/100 + df['TotRmsAbvGrd'] + df['base_bath_tot']
    df['sqr_ft_per_room'] = df['TotalSF']/df['rooms_tot']
    df["quality_add"]=df['KitchenQual']+df['GarageQual']+df['FireplaceQu']+df['HeatingQC']+df['BsmtQual']+df['ExterQual']+df['OverallQual']
    df["condition_add"]=df['KitchenQual']+df['GarageCond']+df['HeatingQC']+df['BsmtCond']+df['ExterCond']+df['OverallCond']
    df['metrics_add'] = df['condition_add']+df['quality_add']
    df["quality_add"]=df['KitchenQual']+df['GarageQual']+df['FireplaceQu']+df['HeatingQC']+df['BsmtQual']+df['ExterQual']+df['OverallQual']
    df["quality_mult"]=df['KitchenQual']*2 + df['GarageCond']*df['GarageQual'] + 2*df['HeatingQC']+df['BsmtQual']*df['BsmtCond']+df['ExterQual']*df['ExterCond']+df['OverallCond']*df['OverallQual']*2
    return df

In [ ]:
tt=feat_engineering(train)

In [ ]:
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(tt, target, test_size=0.33, random_state=42)

In [ ]:
models = { 
    'TTR': TransformedTargetRegressor()}

params_closer = {
    'TTR': {
        'regressor':  [ 
            Ridge(alpha=6),
            Ridge(alpha=7),
            Ridge(alpha=8),
            Ridge(alpha=9),
            Ridge(alpha=10),
            Ridge(alpha=11),
            Ridge(alpha=12),
            Ridge(alpha=13),
            Ridge(alpha=14),
            Ridge(alpha=15),
            Ridge(alpha=16),
            ElasticNet(alpha=0.01),
            ElasticNet(alpha=0.1),
            ElasticNet(alpha=1),
            ElasticNet(alpha=10),
            HuberRegressor(alpha=0.01),
            HuberRegressor(alpha=0.1),
            HuberRegressor(alpha=1),
            HuberRegressor(alpha=10)],
        'transformer': [QuantileTransformer(n_quantiles=750,output_distribution='normal')]},
}

In [ ]:
helper1 = EstimatorSelectionHelper(models, params_closer)
helper1.fit(X_train_full, y_train_full, scoring='r2', n_jobs=1)
helper1.score_summary().head(20)

In [ ]:
helper1.score_summary().params[0]

In [ ]:
regr_trans_full = TransformedTargetRegressor(
    regressor= Ridge(alpha=9),
    transformer=QuantileTransformer(n_quantiles=750,
                                    output_distribution='normal'))

In [ ]:
regr_trans_full = regr_trans_full.fit(X_train_full,y_train_full)
y_pred = regr_trans_full.score(X_test_full,y_test_full)
ttree = regr_trans_full.score(X_train_full, y_train_full)
pred_train = regr_trans_full.predict(X_train_full)
pred_test = regr_trans_full.predict(X_test_full)
print("Training score of Tree Model",y_pred)
print("Testing Score of Tree Model",ttree)
print(f'\nTraining RMSE of model: {rmse(pred_train,y_train_full)}')
print(f'Testing RMSE of model: {rmse(pred_test,y_test_full)}')
print(f'\nTraining MAE of Reg model: {mae(pred_train,y_train_full)}')
print(f'Testing MAE of Reg model: {mae(pred_test,y_test_full)}')

### Genetic Algos

Something I have been working on and very interested in ius GeneticSelection Algorithms
They offer a good chance of automating difficult parts of modeling

In [ ]:
!pip install genetic_selection
from genetic_selection import GeneticSelectionCV
# create the GeneticSelection search with the different parameters available.
selection = GeneticSelectionCV(regr_trans_full,
                              cv=5,
                              scoring="r2",
                              max_features=20,
                              n_population=120,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=50,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              n_gen_no_change=10,
                              n_jobs=-1)

# fit the GA search to our data.
selection = selection.fit(X_train_full, y_train_full)

# print the results.
print(selection.support_)

In [ ]:
newsies=tt.loc[:, selection.support_ == True]

In [ ]:
newsies.columns

In [ ]:
X, X_t, y, y_t = train_test_split(newsies, target, test_size=0.33, random_state=42)

In [ ]:
#just for comparisons sake this is the old baseline model with few features
log3 = LinearRegression()
log3 = log3.fit(X,y)
y_predL = log3.score(X_t,y_t)
ttreeL = log3.score(X, y)
predL = log3.predict(X)
#predNL = np.expm1(predL)
predL_test = log3.predict(X_t)
#predNL_test = np.expm1(predL_test)

print("Training score of Reg Model",ttreeL)
print("Testing Score of Reg Model",y_predL)
print(f'\nTraining RMSE of Reg model: {rmse(predL,y)}')
print(f'Testing RMSE of Reg model: {rmse(predL_test,y_t)}')
print(f'\nTraining MAE of Reg model: {mae(predL,y)}')
print(f'Testing MAE of Reg model: {mae(predL_test,y_t)}')

In [ ]:
helper11 = EstimatorSelectionHelper(models, params_closer)
helper11.fit(X, y, scoring='r2', n_jobs=1)
helper11.score_summary().head(20)

In [ ]:
from sklearn.linear_model import Lasso
alphas=[0.001, 0.1, 2, 4, 6, 8, 10, 12, 14, 50]

models3 = { 
    'Linear': LinearRegression(),
    'Lars': LassoLars(),
    'RFReg': RandomForestRegressor(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'EN': ElasticNet(),
    'Huber': HuberRegressor()}

params3 = {
    'Linear':  {
        'normalize': ['True', 'False']},
    'Lars': {
        'fit_intercept': ['True'],
        'normalize': ['True', 'False'],
        'alpha': alphas,
        'eps': [0,10,20,30,40,50,60,70]},
    'RFReg': {
        'n_estimators': [30, 60, 90],
        'criterion': ['mse', 'mae'],
        'max_depth': [6, 12],
        'min_samples_split': [2, 5],
        'max_features': [3,7,10,'n_features'],
        'bootstrap': ['False']},
    'Lasso': {
        'alpha': [.5,1.0,2,10],
        'fit_intercept': ['False'],
        'normalize': ['False', 'True'],
        'max_iter': [50,500,1000,5000]},
    'Ridge': {"alpha": alphas},
    'EN': {
        'alpha': alphas,
        'l1_ratio': [0.25, 0.5, 0.75]},
    'Huber': {
        'alpha': alphas, 
        'epsilon': [1,1.35,1.9]}
}

In [ ]:
helper11.score_summary().params[0]

In [ ]:
helper1 = EstimatorSelectionHelper(models3, params3)
helper1.fit(X_train3, y_train3, scoring='r2', n_jobs=1)
helper1.score_summary().head(20)

In [ ]:
helper1 = EstimatorSelectionHelper(models, params_closer)
helper1.fit(X, y, scoring='r2', n_jobs=1)
helper1.score_summary().head(20)

In [ ]:
helper1.score_summary().params[0]

In [ ]:
best_fit = TransformedTargetRegressor(
                                regressor= Ridge(alpha=6),
                                transformer=QuantileTransformer(n_quantiles=750,
                                output_distribution='normal'))
best_fit2 = LassoLars(alpha = 10, eps = 0, fit_intercept = True, normalize = False)


In [ ]:
cs_choice = best_fit
cs_choice = cs_choice.fit(X,y)
y_pred_cv_test = cs_choice.score(X_t,y_t)
y_pred_cv_train = cs_choice.score(X, y)
pred_cv_train = cs_choice.predict(X)
pred_cv_test = cs_choice.predict(X_t)
print("Training score of Model",y_pred_cv_train)
print("Testing Score of Model",y_pred_cv_test)
print(f'\nTraining RMSE of model: {rmse(pred_cv_train,y)}')
print(f'Testing RMSE of model: {rmse(pred_cv_test,y_t)}')
print(f'\nTraining MAE of Reg model: {mae(pred_cv_train,y)}')
print(f'Testing MAE of Reg model: {mae(pred_cv_test,y_t)}')

In [ ]:
cs_choice = best_fit2
cs_choice = cs_choice.fit(X,y)
y_pred_cv_test = cs_choice.score(X_t,y_t)
y_pred_cv_train = cs_choice.score(X, y)
pred_cv_train = cs_choice.predict(X)
pred_cv_test = cs_choice.predict(X_t)
print("Training score of Model",y_pred_cv_train)
print("Testing Score of Model",y_pred_cv_test)
print(f'\nTraining RMSE of model: {rmse(pred_cv_train,y)}')
print(f'Testing RMSE of model: {rmse(pred_cv_test,y_t)}')
print(f'\nTraining MAE of Reg model: {mae(pred_cv_train,y)}')
print(f'Testing MAE of Reg model: {mae(pred_cv_test,y_t)}')

In [ ]:
best_fit2.coef_